In [113]:
%env CUDA_LAUNCH_BLOCKING=1
%load_ext autoreload
%autoreload complete
# %aimport ...

import sys
import importlib
import warnings

importlib.invalidate_caches()

for mod_name in list(sys.modules):
    if not (str.startswith(mod_name, "robotodo") or str.startswith(mod_name, "tensorspecs")):
        continue
    try:
        importlib.reload(sys.modules[mod_name])
    except Exception as error:
        warnings.warn(f"{error}")


env: CUDA_LAUNCH_BLOCKING=1
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:

from robotodo.engines.isaac._kernel import Kernel

kernel = Kernel([
    "--/log/file=/tmp/todo-isaacsim.log",
    # "--/log/enableStandardStreamOutput=true",
    "--/app/enableStdoutOutput=true",
    # "--no-window",
    # TODO
    # "--/app/content/emptyStageOnStart=false",
    # "--/app/asyncRendering=true",
    # "--enable", "isaacsim.exp.full",
    "--/app/vulkan=",
], kit_path="isaacsim.exp.full.kit")
kernel.start_app_loop_soon()

[Warning] [simulation_app] Interactive python shell detected but ISAAC_JUPYTER_KERNEL was not set. Problems with asyncio may occur
[Warning] [simulation_app] Please use Isaac Sim Python 3 kernel instead of the default Python 3 Kernel
Loading user config located at: '/home/sysadmin/lab/robotodo/.conda/lib/python3.11/site-packages/omni/data/Kit/Isaac-Sim Full/5.0/user.config.json'
[0.062s] [ext: omni.kit.async_engine-0.0.3] startup
[0.649s] [ext: omni.metrics.core-0.0.3] startup
[0.649s] [ext: omni.client.lib-1.1.0] startup
[0.668s] [ext: omni.blobkey-1.1.2] startup
[0.669s] [ext: omni.stats-1.0.1] startup
[0.670s] [ext: omni.datastore-0.0.0] startup
[0.678s] [ext: omni.client-1.3.0] startup
[0.733s] [ext: omni.ujitso.default-1.0.0] startup
[0.735s] [ext: omni.hsscclient-1.1.2] startup
[0.746s] [ext: omni.gpu_foundation.shadercache.vulkan-1.0.0] startup
[0.749s] [ext: omni.assets.plugins-0.0.0] startup
[0.749s] [ext: omni.gpu_foundation-0.0.0] startup
[0.764s] [ext: carb.windowing.plugin

2025-10-24T10:11:06Z s] [Error] [omni.gpu_foundation_factory.plugin] Start up failed. The default graphics plugin cannot be set!



|---------------------------------------------------------------------------------------------|
| Driver Version: 535.247.01    | Graphics API: Vulkan
|=============================================================================================|
| GPU | Name                             | Active | LDA | GPU Memory | Vendor-ID | LUID       |
|     |                                  |        |     |            | Device-ID | UUID       |
|     |                                  |        |     |            | Bus-ID    |            |
|---------------------------------------------------------------------------------------------|
| 0   | NVIDIA GeForce RTX 3060          | Yes: 0 |     | 12288   MB | 10de      | 0          |
|     |                                  |        |     |            | 2504      | 3157457f.. |
|     |                                  |        |     |            | 1         |            |
|===============================================================================

2025-10-24T10:11:18Z [12,666ms] [Error] [isaacsim.ros2.bridge.impl.extension] ROS2 Bridge startup failed


[13.528s] [ext: omni.kit.registry.nucleus-0.0.0] startup
[13.761s] app ready


2025-10-24T10:11:19Z [13,775ms] [Error] [carb.graphics-vulkan.plugin] Could not get NGX parameters block because NGX isn't enabled.
2025-10-24T10:11:19Z [13,775ms] [Error] [carb.graphics-vulkan.plugin] Failed to create NGX context.
2025-10-24T10:11:19Z [13,870ms] [Error] [rtx.optixdenoising73.plugin] Failed to create an Optix adaptor for device 0


In [ ]:
# TODO 
from robotodo.engines.isaac.builders import load_usd_scene

# TODO set up axis
scene = await load_usd_scene(
    "https://omniverse-content-production.s3-us-west-2.amazonaws.com/Assets/Isaac/5.0/Isaac/Environments/Simple_Room/simple_room.usd",
    # "https://omniverse-content-production.s3-us-west-2.amazonaws.com/Assets/Isaac/5.0/Isaac/Environments/Grid/default_environment.usd",
    # "https://omniverse-content-production.s3-us-west-2.amazonaws.com/Assets/Isaac/5.0/Isaac/Environments/Grid/gridroom_black.usd",
    _kernel=kernel,
)

In [188]:
from robotodo.engines.isaac.entity import Entity

In [189]:
from robotodo.engines.isaac.scene import Scene

scene = Scene(_kernel=kernel)

In [ ]:
# entity.material
# entity.material.density

## TODO: material

In [ ]:
# TODO
# https://docs.omniverse.nvidia.com/kit/docs/omni_physics/latest/dev_guide/deformables_beta/deformable_beta.html#create-and-assign-materials

import functools
from typing import Callable

# TODO
import numpy

from robotodo.engines.isaac._utils import USDPrimHelper

from robotodo.engines.core.path import PathExpression, PathExpressionLike


class Material:

    _usd_prims_ref: Callable[[Scene], "list[pxr.Usd.Prim]"]

    def __init__(
        self,
        ref: PathExpressionLike | Callable[[Scene], "list[pxr.Usd.Prim]"],
        scene: Scene,
    ):
        match ref:
            case _ if isinstance(ref, Callable):
                self._usd_prims_ref = ref
            case _:
                # TODO cache
                class _USDPrimsRef:
                    def __init__(self, path):
                        self._path = path
                    
                    def __repr__(self):
                        return repr(self._path)
                    
                    def __call__(self, scene: Scene):
                        pxr = scene._kernel.pxr
                        return [
                            maybe_material_prim
                            for prim_path in scene.resolve(self._path)
                            for maybe_material_prim in pxr.Usd.PrimRange(
                                scene._usd_stage.GetPrimAtPath(prim_path), 
                                # pxr.Usd.TraverseInstanceProxies(
                                #     pxr.Usd.PrimAllPrimsPredicate
                                # ),
                            )
                            if maybe_material_prim.IsA(pxr.UsdShade.Material)
                        ]
                self._usd_prims_ref = _USDPrimsRef(ref)
        # TODO maybe _kernel?
        self._scene = scene

    def __repr__(self):
        return f"{self.__class__.__qualname__}({self._usd_prims_ref!r}, scene={self._scene})"

    @property
    def _usd_prims(self):
        return self._usd_prims_ref(self._scene)
    
    @property
    def path(self):
        return [
            material_prim.GetPath().pathString
            if material_prim else
            None
            for material_prim in self._usd_prims
        ]

    @property
    def static_friction(self):
        pxr = self._scene._kernel.pxr

        # TODO FIXME perf
        res = []

        for idx, material_prim in enumerate(self._usd_prims):
            value = numpy.nan

            if not material_prim:
                warnings.warn(
                    f"USD Prim at index {idx} is invalid: {material_prim}"
                )
            else:
                if material_prim.HasAPI(pxr.UsdPhysics.MaterialAPI):
                    value = (
                        pxr.UsdPhysics.MaterialAPI(material_prim)
                        .GetStaticFrictionAttr()
                        .Get()
                    )
                if material_prim.HasAPI("OmniPhysicsBaseMaterialAPI"):
                    value = material_prim.GetAttribute("omniphysics:staticFriction").Get()

            res.append(value)

        return numpy.asarray(res)

    @static_friction.setter
    def static_friction(self, value):
        pxr = self._scene._kernel.pxr

        material_prims = self._usd_prims

        for idx, (material_prim, v) in enumerate(zip(
            material_prims,
            numpy.broadcast_to(value, len(material_prims)),
            strict=True,
        )):
            if not material_prim:
                warnings.warn(
                    f"USD Prim at index {idx} is invalid: {material_prim}"
                )
            # TODO
            pxr.UsdPhysics.MaterialAPI.Apply(material_prim) \
                .GetStaticFrictionAttr().Set(float(v))
            # TODO
            if material_prim.HasAPI("OmniPhysicsBaseMaterialAPI"):
                material_prim.GetAttribute("omniphysics:staticFriction").Set(float(v))

    # TODO
    _ = """
    # TODO
    legacy_api = pxr.UsdPhysics.MaterialAPI.Apply(todo_material)
    print(legacy_api.CreateDynamicFrictionAttr().Get())
    todo_material.ApplyAPI("OmniPhysicsBaseMaterialAPI")
    print(todo_material.GetAttribute("omniphysics:dynamicFriction").Get())

    # TODO
    legacy_api = pxr.UsdPhysics.MaterialAPI.Apply(todo_material)
    print(legacy_api.CreateDensityAttr().Get())
    todo_material.ApplyAPI("OmniPhysicsBaseMaterialAPI")
    print(todo_material.GetAttribute("omniphysics:density").Get())
    """

    # TODO
    _ = """
    # TODO
    prim.ApplyAPI("OmniPhysicsDeformableMaterialAPI")
    prim.GetAttribute("omniphysics:youngsModulus").Set(youngs)
    prim.GetAttribute("omniphysics:poissonsRatio").Set(poissons)

    # TODO
    prim.ApplyAPI("OmniPhysicsSurfaceDeformableMaterialAPI")
    prim.GetAttribute("omniphysics:surfaceThickness").Set(thickness)
    # Estimate bend stiffness from Young's modulus and Poisson's ratio
    # surfaceThickness is incorporated in PhysX
    bendStiffness = youngs/(12.0*(1.0-poissons**2))
    prim.GetAttribute("omniphysics:surfaceBendStiffness").Set(bendStiffness)
    """


def _todo_Entity_material(self: Entity):
    pxr = self._scene._kernel.pxr

    def _materials_ref(scene: Scene):
        materials, _ = pxr.UsdShade.MaterialBindingAPI.ComputeBoundMaterials(self._usd_prims)
        # TODO ensure valid
        return [material.GetPrim() for material in materials]

    return Material(_materials_ref, scene=self._scene)


def _todo_Entity_material_setter(self: Entity, material: Material):
    pxr = self._scene._kernel.pxr

    prims, material_prims = numpy.broadcast_arrays(self._usd_prims, material._usd_prims)

    for prim, material_prim in zip(prims, material_prims, strict=True):
        material_binding = pxr.UsdShade.MaterialBindingAPI.Apply(prim)
        material_binding.Bind(
            pxr.UsdShade.Material(material_prim), 
            # TODO
            pxr.UsdShade.Tokens.strongerThanDescendants, # pxr.UsdShade.Tokens.weakerThanDescendants, 
            # "physics",
        )



In [292]:
Material("/**/*", scene=scene).path

['/Root/Looks/ParquetFloor',
 '/Root/Looks/WhitePlastic',
 '/Root/Looks/WhitePlastic2',
 '/Root/Looks/WoodWall',
 '/Root/Looks/Glass',
 '/Root/Looks/Light',
 '/Root/Looks/Table',
 '/Root/Looks/ParquetFloor',
 '/Root/Looks/WhitePlastic',
 '/Root/Looks/WhitePlastic2',
 '/Root/Looks/WoodWall',
 '/Root/Looks/Glass',
 '/Root/Looks/Light',
 '/Root/Looks/Table',
 '/World/Dress/Looks/Cotton_50s_Poplin_FRONT_83472',
 '/World/Dress/Material/linen_Blue',
 '/World/Dress/Looks/Cotton_50s_Poplin_FRONT_83472',
 '/World/Dress/Looks/Cotton_50s_Poplin_FRONT_83472',
 '/World/Dress/Material/linen_Blue',
 '/World/Dress/Material/linen_Blue']

In [301]:
_todo_Entity_material(Entity("/**/*", scene=scene)).static_friction

/tmp/ipykernel_486207/1348015942.py:97: UserWarning: USD Prim at index 0 is invalid: invalid null prim
  warnings.warn(
/tmp/ipykernel_486207/1348015942.py:97: UserWarning: USD Prim at index 1 is invalid: invalid null prim
  warnings.warn(
/tmp/ipykernel_486207/1348015942.py:97: UserWarning: USD Prim at index 2 is invalid: invalid null prim
  warnings.warn(
/tmp/ipykernel_486207/1348015942.py:97: UserWarning: USD Prim at index 3 is invalid: invalid null prim
  warnings.warn(
/tmp/ipykernel_486207/1348015942.py:97: UserWarning: USD Prim at index 4 is invalid: invalid null prim
  warnings.warn(
/tmp/ipykernel_486207/1348015942.py:97: UserWarning: USD Prim at index 5 is invalid: invalid null prim
  warnings.warn(
/tmp/ipykernel_486207/1348015942.py:97: UserWarning: USD Prim at index 6 is invalid: invalid null prim
  warnings.warn(
/tmp/ipykernel_486207/1348015942.py:97: UserWarning: USD Prim at index 7 is invalid: invalid null prim
  warnings.warn(
/tmp/ipykernel_486207/1348015942.py:97: 

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        1., nan,  1., nan,  1., nan,  1., nan,  1., nan,  1., nan,  1.,
       nan,  1., nan,  1., nan,  1., nan,  1., nan,  1., nan,  1., nan,
        1., nan,  1., nan,  1., nan,  1., nan,  1., nan,  1., nan,  1.,
       nan,  1., nan,  1., nan,  1., nan,  1., nan,  1., nan,  1., nan,
        1., nan,  1., nan,  1., nan,  1., nan,  1., nan,  1., nan,  1.,
       nan,  1., nan,  1., nan,  1., nan,  1., nan,  1., nan,  1., nan,
        1., nan,  1., nan,  1., nan,  1., nan,  1., nan,  1., nan,  1.,
       nan,  1., nan,  1., nan,  1., nan,  1., nan,  1., nan,  1., nan,
        1., nan,  1., nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.])

In [303]:
entity = Entity("/**/*", scene=scene)

# _todo_Entity_material_setter(entity, _todo_Entity_material(entity))

In [304]:
# TODO mv Entity.material
_todo_Entity_material(Entity("/**/*", scene=scene)).path

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 '/Root/Looks/ParquetFloor',
 None,
 '/Root/Looks/WhitePlastic2',
 None,
 '/Root/Looks/WhitePlastic2',
 None,
 '/Root/Looks/WhitePlastic2',
 None,
 '/Root/Looks/WhitePlastic2',
 None,
 '/Root/Looks/WhitePlastic2',
 None,
 '/Root/Looks/WhitePlastic2',
 None,
 '/Root/Looks/WhitePlastic2',
 None,
 '/Root/Looks/WhitePlastic2',
 None,
 '/Root/Looks/WhitePlastic2',
 None,
 '/Root/Looks/WhitePlastic2',
 None,
 '/Root/Looks/WhitePlastic2',
 None,
 '/Root/Looks/WhitePlastic2',
 None,
 '/Root/Looks/WhitePlastic2',
 None,
 '/Root/Looks/WhitePlastic2',
 None,
 '/Root/Looks/WhitePlastic2',
 None,
 '/Root/Looks/WhitePlastic2',
 None,
 '/Root/Looks/WhitePlastic2',
 None,
 '/Root/Looks/WhitePlastic2',
 None,
 '/Root/Looks/WhitePlastic2',
 None,
 '/Root/Looks/WhitePlastic2',
 None,
 '/Root/Looks/WhitePlastic2',
 None,
 '/Ro

In [278]:
_todo_Entity_material(Entity("/**/*", scene=scene)).static_friction

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [279]:
Material("/World/Dress/Material", scene=scene).static_friction

array([1.])

In [239]:
Material("/**/*", scene=scene).path

['/Root/Looks/Glass',
 '/Root/Looks/WoodWall',
 '/Root/Looks/WhitePlastic',
 '/Root/Looks/Light',
 '/Root/Looks/ParquetFloor',
 '/World/Dress/Material/linen_Blue',
 '/World/Dress/Looks/Cotton_50s_Poplin_FRONT_83472',
 '/Root/Looks/Table',
 '/Root/Looks/WhitePlastic2']

In [ ]:
import pxr

stage = scene._usd_stage
todo_material = stage.GetPrimAtPath("/World/Dress/Material/linen_Blue")
# TODO
todo_material.GetAppliedSchemas()

# TODO
legacy_api = pxr.UsdPhysics.MaterialAPI.Apply(todo_material)
print(legacy_api.CreateStaticFrictionAttr().Get())

todo_material.ApplyAPI("OmniPhysicsBaseMaterialAPI")
print(todo_material.GetAttribute("omniphysics:staticFriction").Get())

# TODO
legacy_api = pxr.UsdPhysics.MaterialAPI.Apply(todo_material)
print(legacy_api.CreateDynamicFrictionAttr().Get())

todo_material.ApplyAPI("OmniPhysicsBaseMaterialAPI")
print(todo_material.GetAttribute("omniphysics:dynamicFriction").Get())

# TODO
legacy_api = pxr.UsdPhysics.MaterialAPI.Apply(todo_material)
print(legacy_api.CreateDensityAttr().Get())

todo_material.ApplyAPI("OmniPhysicsBaseMaterialAPI")
print(todo_material.GetAttribute("omniphysics:density").Get())

1.0
10.0
3.6000001430511475
100.0
1.0
0.0


In [ ]:
import omni
omni.physx.bindings._physx.hasconflictingapis_DeformableBodyAPI(todo_material, True)

True

In [ ]:
{p: p.Get() for p in todo_material.GetProperties()}

{Usd.Prim(</World/Dress/Material/linen_Blue>).GetAttribute('omniphysics:density'): 0.0,
 Usd.Prim(</World/Dress/Material/linen_Blue>).GetAttribute('omniphysics:dynamicFriction'): 100.0,
 Usd.Prim(</World/Dress/Material/linen_Blue>).GetAttribute('omniphysics:staticFriction'): 1.0,
 Usd.Prim(</World/Dress/Material/linen_Blue>).GetAttribute('outputs:displacement'): None,
 Usd.Prim(</World/Dress/Material/linen_Blue>).GetAttribute('outputs:surface'): None,
 Usd.Prim(</World/Dress/Material/linen_Blue>).GetAttribute('outputs:volume'): None,
 Usd.Prim(</World/Dress/Material/linen_Blue>).GetAttribute('paused'): False,
 Usd.Prim(</World/Dress/Material/linen_Blue>).GetAttribute('physics:density'): 1.0,
 Usd.Prim(</World/Dress/Material/linen_Blue>).GetAttribute('physics:dynamicFriction'): 3.6000001430511475,
 Usd.Prim(</World/Dress/Material/linen_Blue>).GetAttribute('physics:restitution'): 0.0,
 Usd.Prim(</World/Dress/Material/linen_Blue>).GetAttribute('physics:staticFriction'): 1.0}

In [ ]:
# entity.material = Material(...)

In [173]:
# floor = Entity("/Root/Floor", scene=scene)
scene._isaac_physx_simulation.flush_changes()
todo = scene._isaac_physics_tensor_view.create_rigid_body_view("/*")
todo.get_material_properties()

tensor([[[1.0000, 3.6000, 0.0000]]])

In [179]:
import torch

todo.set_velocities(torch.asarray([[1, 1, 1, 0, 0, 0]]), indices=torch.asarray([0]))

In [180]:
todo.get_velocities()

tensor([[-0.0011,  0.0006, -0.0011, -0.0002, -0.0016, -0.0004]])

In [192]:
todo_ = scene._isaac_physics_tensor_view.create_surface_deformable_body_view("/World/Dress")
# scene._isaac_physics_tensor_view.create_volume_deformable_body_view("/World/Dress")
todo_.collision_mesh_prim_paths
todo_.get_transforms()
# todo_.get_simulation_nodal_positions()
# todo_.get_simulation_nodal_velocities()

tensor([[0.3339, 0.3162, 1.0629, 0.0000, 0.0000, 0.0000, 1.0000]])

In [243]:
# scene._isaac_physx_simulation.flush_changes()
# todo_deformable = scene._isaac_physics_tensor_view.create_deformable_material_view("/World/Dress/Material/linen_Blue")
# todo_deformable.get_dynamic_friction()

In [ ]:

def create_volume_material(stage, path, youngs, poissons):
    mat = UsdShade.Material.Define(stage, path)
    prim = mat.GetPrim()

    # Common parameters
    prim.ApplyAPI("OmniPhysicsBaseMaterialAPI")
    prim.GetAttribute("omniphysics:staticFriction").Set(10.0)
    prim.GetAttribute("omniphysics:dynamicFriction").Set(10.0)
    prim.GetAttribute("omniphysics:density").Set(1000.0)

    # Common deformable parameters
    prim.ApplyAPI("OmniPhysicsDeformableMaterialAPI")
    prim.GetAttribute("omniphysics:youngsModulus").Set(youngs)
    prim.GetAttribute("omniphysics:poissonsRatio").Set(poissons)

    # PhysX extras (optional)
    prim.ApplyAPI("PhysxDeformableMaterialAPI")
    prim.GetAttribute("physxDeformableMaterial:elasticityDamping").Set(0.0)
    return mat


def create_surface_material(stage, path, youngs, poissons, thickness):
    mat = UsdShade.Material.Define(stage, path)
    prim = mat.GetPrim()

    # base + surface specific schema APIs
    prim.ApplyAPI("OmniPhysicsBaseMaterialAPI")
    prim.GetAttribute("omniphysics:dynamicFriction").Set(0.2)
    prim.GetAttribute("omniphysics:density").Set(100.0)

    prim.ApplyAPI("OmniPhysicsDeformableMaterialAPI")
    prim.GetAttribute("omniphysics:youngsModulus").Set(youngs)
    prim.GetAttribute("omniphysics:poissonsRatio").Set(poissons)

    prim.ApplyAPI("OmniPhysicsSurfaceDeformableMaterialAPI")
    prim.GetAttribute("omniphysics:surfaceThickness").Set(thickness)

    # Estimate bend stiffness from Young's modulus and Poisson's ratio
    # surfaceThickness is incorporated in PhysX
    bendStiffness = youngs/(12.0*(1.0-poissons**2))
    prim.GetAttribute("omniphysics:surfaceBendStiffness").Set(bendStiffness)

    # PhysX extras
    prim.ApplyAPI("PhysxSurfaceDeformableMaterialAPI")
    prim.GetAttribute("physxDeformableMaterial:elasticityDamping").Set(0.0)
    prim.GetAttribute("physxDeformableMaterial:bendDamping").Set(0.0)
    return mat




In [ ]:
from robotodo.engines.isaac.builders import *
from robotodo.engines.isaac.builders import _usd_get_or_use_free_path


class MaterialBuilder:
    class Config(TypedDict):
        path: Optional[PathExpressionLike]
        # TODO
        # replace: Optional[bool]

    async def __call__(
        self,
        scene: Scene,
        config: Config = Config(),
    ):

        pxr = scene._kernel.pxr
        scene._kernel.import_module("pxr.UsdShade")

        stage = scene._usd_stage
        prim_path = _usd_get_or_use_free_path(
            scene=scene, 
            path=config.get("path", None),
        )

        prim = pxr.UsdShade.Material.Define(stage, prim_path)

        # TODO FIXME entity: no path roundtrip; ref underlying prim directly
        return Material(path=prim.GetPath().pathString, scene=scene)

<module 'omni.physx.scripts.deformableUtils' from '/home/sysadmin/lab/robotodo/.conda/lib/python3.11/site-packages/omni/data/Kit/Isaac-Sim Full/5.0/exts/3/omni.physx-107.3.18+107.3.1.lx64.r.cp311.u353/omni/physx/scripts/deformableUtils.py'>

In [ ]:
import pxr


visual_material = await load_usd(
    "/home/sysadmin/lab/DexGarmentLab/Assets/Material/Garment/linen_Pumpkin.usda", 
    scene=scene,
    path="/World/Dress/Material",
    replace=True,
)

material_binding = pxr.UsdShade.MaterialBindingAPI.Apply(dress._usd_prims[0])
maybe_material_prim = visual_material._usd_prims[0]

# NOTE the first child_prim would be the prim itself
for material_prim in pxr.Usd.PrimRange(
    maybe_material_prim, 
    pxr.Usd.TraverseInstanceProxies(
        pxr.Usd.PrimAllPrimsPredicate
    ),
):
    if not material_prim.IsA(pxr.UsdShade.Material):
        continue

    material_binding.Bind(
        pxr.UsdShade.Material(material_prim), 
        # TODO
        pxr.UsdShade.Tokens.strongerThanDescendants, # pxr.UsdShade.Tokens.weakerThanDescendants, 
        # "physics",
    )


## TODO: deformable body

### build

In [178]:
from robotodo.engines.isaac.builders import load_usd

dress = await load_usd(
    "/home/sysadmin/lab/DexGarmentLab/Assets/Garment/Dress/Long_LongSleeve/DLLS_Dress008_0/DLLS_Dress008_0_obj.usd", 
    scene=scene,
    path="/World/Dress",
    replace=True,
)


2025-10-27T05:52:15Z [243,670,009ms] [Error] [omni.usd.commands.usd_commands] /World/Dress does not exist


In [93]:
import numpy

from robotodo.engines.isaac.entity import *
from robotodo.engines.core.entity import EntityBodyKind


class _TODONextEntity(Entity):

    @property
    def body_kind(self):
        res = []

        for prim in self._usd_prims:
            v = EntityBodyKind.NONE

            for schema in prim.GetAppliedSchemas():
                match schema:
                    case "PhysicsRigidBodyAPI":
                        if (
                            pxr.UsdPhysics.RigidBodyAPI(prim)
                            .GetRigidBodyEnabledAttr()
                            .Get()
                        ):
                            v = EntityBodyKind.RIGID
                    case "OmniPhysicsVolumeDeformableSimAPI":
                        if (
                            prim
                            .GetAttribute("omniphysics:deformableBodyEnabled")
                            .Get()
                        ):
                            v = EntityBodyKind.DEFORMABLE_VOLUME
                    case "OmniPhysicsSurfaceDeformableSimAPI":
                        if (
                            prim
                            .GetAttribute("omniphysics:deformableBodyEnabled")
                            .Get()
                        ):
                            v = EntityBodyKind.DEFORMABLE_SURFACE

            res.append(v.value)

        return numpy.asarray(res)

    @body_kind.setter
    def body_kind(self, value):
        
        omni = self._scene._kernel.omni
        self._scene._kernel.enable_extension("omni.physx")
        self._scene._kernel.import_module("omni.physx.scripts")

        prims = self._usd_prims

        for prim, v in zip(
            prims,
            numpy.broadcast_to(value, len(prims)),
            strict=True,
        ):
            match v:
                case EntityBodyKind.NONE:
                    omni.physx.scripts.deformableUtils.remove_deformable_body(
                        scene._usd_stage, 
                        prim_path=prim.GetPath(),
                    )
                    omni.physx.scripts.utils.removeRigidBody(prim)
                case EntityBodyKind.RIGID:
                    omni.physx.scripts.deformableUtils.remove_deformable_body(
                        scene._usd_stage, 
                        prim_path=prim.GetPath(),
                    )
                    omni.physx.scripts.utils.setRigidBody(
                        prim, 
                        approximationShape=None, 
                        kinematic=False,
                    )
                case EntityBodyKind.DEFORMABLE_VOLUME:
                    omni.physx.scripts.utils.removeRigidBody(prim)
                    # TODO check
                    omni.physx.scripts.deformableUtils.set_physics_volume_deformable_body(
                        scene._usd_stage, 
                        prim_path=prim.GetPath(),
                    )
                case EntityBodyKind.DEFORMABLE_SURFACE:
                    omni.physx.scripts.utils.removeRigidBody(prim)
                    # TODO check
                    omni.physx.scripts.deformableUtils.set_physics_surface_deformable_body(
                        scene._usd_stage, 
                        prim_path=prim.GetPath(),
                    )


In [111]:
from omni.physx import acquire_physx_cooking_interface

physx_cooking = acquire_physx_cooking_interface()
physx_cooking.compute_conforming_tetrahedral_mesh?

Docstring:
compute_conforming_tetrahedral_mesh(self: omni.physx.bindings._physx.PhysXCooking, src_tri_points: List[carb._carb.Float3], src_tri_indices: List[int]) -> dict


Create a conforming tetrahedral mesh from a closed source triangle mesh.

Args:
    src_tri_points      Vertices of the source triangle mesh
    src_tri_indices     Vertex indices of the source triangles

Returns:
    Dict mapping 'points' and 'indices' to the resulting tetrahedral mesh points and indices

The conforming tetrahedral mesh is defined as a tetrahedral mesh whose surface triangles align with the closed source triangle mesh and whose
internal vertices lie on the inside of the closed source triangle mesh.
Type:      method

In [147]:
todo_mesh_geom = Entity("/World/Dress", scene=scene).geometry[0][0]

In [150]:
# TODO
# todo_mesh_geom.triangulate()
todo_trimesh = todo_mesh_geom.to_triangular()
todo_trimesh.vertices


array([[-0.41878307,  1.3082724 , -0.00148968],
       [-0.404436  ,  1.31109985, -0.00159848],
       [-0.39007957,  1.31388458, -0.00168416],
       ...,
       [-0.05647657,  0.9465435 ,  0.1084809 ],
       [-0.06553775,  0.93325218,  0.11097391],
       [-0.04847033,  0.95138969,  0.10888946]])

In [128]:
todo_trimesh.vertices.shape

(8060, 3)

In [131]:
(todo_trimesh.face_vertex_indices)

[array([605, 609, 606], dtype=int32),
 array([ 0, 47, 71], dtype=int32),
 array([110,  88,  87], dtype=int32),
 array([ 71,  88, 111], dtype=int32),
 array([110,   3,   2], dtype=int32),
 array([737, 723, 724], dtype=int32),
 array([ 53, 149,  52], dtype=int32),
 array([304, 374, 306], dtype=int32),
 array([338, 337, 176], dtype=int32),
 array([140, 200, 109], dtype=int32),
 array([139,   5,   4], dtype=int32),
 array([104,  82,  79], dtype=int32),
 array([596,  92, 707], dtype=int32),
 array([202,   6,   5], dtype=int32),
 array([138, 197, 198], dtype=int32),
 array([69, 70, 82], dtype=int32),
 array([202, 109, 201], dtype=int32),
 array([  2,   1, 111], dtype=int32),
 array([ 1,  0, 71], dtype=int32),
 array([377, 312, 161], dtype=int32),
 array([415, 285, 364], dtype=int32),
 array([113, 112,  48], dtype=int32),
 array([198,   7,   6], dtype=int32),
 array([ 71, 111,   1], dtype=int32),
 array([140, 110,  87], dtype=int32),
 array([292, 211, 373], dtype=int32),
 array([110, 111,  88

In [149]:
import numpy

res = physx_cooking.compute_conforming_tetrahedral_mesh(
    src_tri_points=todo_trimesh.vertices,
    src_tri_indices=numpy.reshape(todo_trimesh.face_vertex_indices, -1),
)
# res["points"]
# res["indices"]

In [167]:
import pxr

stage = scene._usd_stage
tetmesh = pxr.UsdGeom.TetMesh.Define(stage, "/TODOTetMesh")
tetmesh.CreatePointsAttr().Set(
    numpy.asarray(res["points"])
)
tetmesh.CreateTetVertexIndicesAttr().Set(
    numpy.reshape(res["indices"], (-1, 4))
)
tetmesh.CreateOrientationAttr().Set("rightHanded")

tetmesh.CreateSurfaceFaceVertexIndicesAttr().Set(
    pxr.UsdGeom.TetMesh.ComputeSurfaceFaces(tetmesh)
)

True

In [170]:
from pxr import Usd, UsdGeom, Vt, Gf

def create_simulation_tetmesh(stage, path, add_surface=False):

    # Create a tet mesh prim
    tetMesh = UsdGeom.TetMesh.Define(stage, path)

    # Set the points
    points = Vt.Vec3fArray([
        (0, 0, 0), (1, 0, 0), (1, 1, 0), (0, 1, 0),
        (0, 0, 1), (1, 0, 1), (1, 1, 1), (0, 1, 1),
    ])

    # Define three tetrahedra
    tetVtxIndices = Vt.Vec4iArray([
        (0, 1, 3, 4), (1, 2, 3, 6), (1, 4, 5, 6), (3, 4, 6, 7), (1, 3, 4, 6),
    ])

    tetMesh.GetPointsAttr().Set(points)
    tetMesh.GetTetVertexIndicesAttr().Set(tetVtxIndices)

    # Adding surface information for collision detection
    if add_surface:
        surfaceFaceVertexIndices = UsdGeom.TetMesh.ComputeSurfaceFaces(tetMesh)
        tetMesh.GetSurfaceFaceVertexIndicesAttr().Set(surfaceFaceVertexIndices)

    return tetMesh

create_simulation_tetmesh(stage, "/TODO", add_surface=True)


UsdGeom.TetMesh(Usd.Prim(</TODO>))

In [169]:
pxr.UsdGeom.Gprim(tetmesh.GetPrim()).CreateDisplayColorAttr().Set([pxr.Gf.Vec3f(0.7, 0.8, 1.0)])

True

In [168]:
numpy.asarray(res["points"])

array([[-0.41878313,  1.30827212, -0.0014897 ],
       [-0.4044359 ,  1.31110013, -0.0015989 ],
       [-0.39007965,  1.31388509, -0.00168412],
       ...,
       [ 0.10837597,  1.07141852,  0.08833545],
       [-0.02344161,  1.01564562,  0.0226776 ],
       [ 0.08150166,  1.10080612,  0.09086437]])

In [161]:
numpy.asarray(res["points"]).shape
numpy.reshape(res["indices"], (-1, 4)).shape

(24098, 4)

(8164, 3)

In [141]:
# physx_cooking.compute_conforming_tetrahedral_mesh

<bound method PyCapsule.compute_conforming_tetrahedral_mesh of <omni.physx.bindings._physx.PhysXCooking object at 0x7f37fe9d3130>>

In [153]:
len(res["points"]), len(res["indices"])

(8164, 96392)

In [94]:
scene = Scene(_kernel=kernel)

In [95]:
todo_entity = _TODONextEntity("/World/Dress/mesh", scene=scene)
todo_entity.pose

Pose(p=array([[0., 0., 0.]]), q=array([[0.70710678, 0.        , 0.        , 0.70710678]]))

In [102]:
todo_entity.pose *= Pose(p=[0, 0, +.1])

In [100]:
todo_entity.pose = Pose()

In [105]:
todo_entity.body_kind

array([-1])

In [101]:
todo_entity.body_kind = EntityBodyKind.RIGID

In [97]:
todo_entity.body_kind = EntityBodyKind.DEFORMABLE_SURFACE

In [ ]:
# TODO see https://simulately.wiki/docs/snippets/isaac-sim/deformable
# TODO see https://github.com/isaac-sim/IsaacSim/blob/47d886f2858d1ceed556b21c88927aa67bc81c12/source/extensions/isaacsim.core.experimental.prims/python/impl/deformable_prim.py#L131

In [330]:
scene._isaac_physics_tensor_view.create_surface_deformable_body_view("/World/Dress/mesh")

2025-10-24T09:06:50Z [179,629,539ms] [Error] [omni.physx.tensors.plugin] Pattern '/World/Dress/mesh' did not match any surface deformable bodies



In [49]:

# TODO
root_prim_path = dress.path[0]


stage = scene._usd_stage
omni.physx.scripts.deformableUtils.create_auto_surface_deformable_hierarchy(
    stage,
    root_prim_path=root_prim_path,
    simulation_mesh_path=f"{root_prim_path}/SimulationMesh",
    # TODO
    cooking_src_mesh_path=f"{root_prim_path}/mesh",
    #
    cooking_src_simplification_enabled=False,
    set_visibility_with_guide_purpose=True,
)

True

In [186]:
# TODO vol deformable


stage = scene._usd_stage

import pxr

# TODO mesh
src_mesh_prim = stage.GetPrimAtPath("/World/Dress/mesh")
dest_xform_prim = pxr.UsdGeom.Xform.Define(stage, "/World/Dress").GetPrim()

success = omni.physx.scripts.deformableUtils.create_auto_volume_deformable_hierarchy(
    stage=stage,
    root_prim_path=dest_xform_prim.GetPath(),
    simulation_tetmesh_path=dest_xform_prim.GetPath().AppendChild("todosimmesh"),
    simulation_hex_mesh_enabled=True,
    # TODO diff?
    collision_tetmesh_path=dest_xform_prim.GetPath().AppendChild("todocollmesh"),
    cooking_src_mesh_path=src_mesh_prim.GetPath(),
    cooking_src_simplification_enabled=True,
    set_visibility_with_guide_purpose=True,
)
success


True

2025-10-27T05:58:18Z [244,032,379ms] [Error] [omni.physicsschema.plugin] VolumeDeformableSimAPI/SurfaceDeformableSimAPI found that is neither on prim with UsdPhysicsDeformableBody nor an immediate child, (/World/Dress).
2025-10-27T05:58:18Z [244,032,385ms] [Error] [omni.physx.plugin] Deformable volume mesh, failed to setup cooking params, prim: /World/Dress
2025-10-27T05:58:18Z [244,032,385ms] [Error] [omni.physicsschema.plugin] VolumeDeformableSimAPI/SurfaceDeformableSimAPI found that is neither on prim with UsdPhysicsDeformableBody nor an immediate child, (/World/Dress).
2025-10-27T05:58:18Z [244,032,386ms] [Error] [omni.physx.plugin] Deformable volume mesh, failed to setup cooking params, prim: /World/Dress
2025-10-27T05:58:18Z [244,032,387ms] [Error] [omni.physicsschema.plugin] VolumeDeformableSimAPI/SurfaceDeformableSimAPI found that is neither on prim with UsdPhysicsDeformableBody nor an immediate child, (/World/Dress).
2025-10-27T05:58:18Z [244,032,388ms] [Error] [omni.physx.plu

In [184]:
from omni.physx import get_physx_cooking_interface
get_physx_cooking_interface().cook_auto_deformable_body(str(dest_xform_prim.GetPath()))

True

In [172]:
dest_xform_prim.GetPath().AppendChild()

AttributeError: 'Path' object has no attribute 'Append'

In [ ]:
import pxr
pxr.UsdGeom.Xform

### view

In [ ]:
# from curobo.geom.types import WorldConfig, Cuboid, Mesh, Capsule, Cylinder, Sphere

In [106]:
Entity("/World/Dress", scene=scene).viewer.show()